for one piece of the experiment, we have four pointclouds and a .cvs contains position, rotation and grasp_position
step1. let 3 camera rgb and depth images same number
step2. let image and action same number
step3. first appear 1 (from 0), select the image and action which 5 images before the timestep.
step4. add grasp position to .csv
step5. select the corresponding images
step6. try sam2 to do the segmentation get the mask and apply the mask
step6. create the pointcloud (4 pointcloud in total)
(step7. crop the pointcloud)
so the dataset i sent to leo: for one experiment, .csv + 4 pointcloud*2
model: pointnet++ & pointbert -- try on segmented pointcloud and croped pointcloud (single view and merged view)


In [ ]:
'''
import os
import pandas as pd
import glob
import numpy as np
import open3d as o3d

# Path to the base directory where the subfolders are located
base_path = "/media/ubb/4T/human_demonstration/arif_checked_process1"

# Collect all paths for 'tracker_right_zed_modified.csv' and 'tracker_left_zed_modified.csv'
csv_files = glob.glob(os.path.join(base_path, "**/*zed_modified.csv"), recursive=True)

# Prepare a DataFrame to store all changes
all_changes = pd.DataFrame(columns=['name', 'subname', 'time', 'pc_time', 'position', 'orientation', 'l_or_r','grasp_position'])

# Convert position dictionary to numpy array
def position_to_nparray(position):
    return np.array([position['x'], position['y'], position['z']])

# Convert quaternion dictionary to numpy array
def quaternion_to_nparray(quaternion):
    return np.array([quaternion['w'], quaternion['x'], quaternion['y'], quaternion['z']])

# Function to calculate the extended point
def calculate_extended_point(position, quaternion, extension_vector):
    position_np = position_to_nparray(position)
    quaternion_np = quaternion_to_nparray(quaternion)
    R = o3d.geometry.get_rotation_matrix_from_quaternion(quaternion_np)
    # extension_vector = np.array([extension_length_x, extension_length_y, extension_length_z])
    extended_vector = np.dot(R, extension_vector)
    extended_point = position_np + extended_vector
    return extended_point.tolist()

# Function to find every occurrence of a change from 0 to 1 in the 'gripper' column and collect the data
def find_gripper_changes(csv_path):
    # Extract the folder and subfolder name
    parts = csv_path.split(os.sep)
    print(parts)
    name = parts[-5]  # Adjust if necessary
    subname = parts[-4]  # Adjust if necessary
    side = 'right' if 'right' in os.path.basename(csv_path) else 'left'
    
    try:
        # Read the CSV file
        data = pd.read_csv(csv_path)
        data['position'] = data['position'].apply(eval)
        data['orientation'] = data['orientation'].apply(eval)
        
        # Track the last gripper state to identify changes from 0 to 1
        last_gripper = 0
        for index, row in data.iterrows():
            if row['gripper'] == 1 and last_gripper == 0:
                row = data.iloc[index]
                if index >=5:
                    prev_row = data.iloc[index - 5]
                extension_vector = np.array([0.02, 0.045, 0.10]) if side == 'right' else np.array([-0.02, 0.045, 0.10])
                grasp_position = calculate_extended_point(
                        row['position'],
                        row['orientation'],
                        extension_vector
                    )
                all_changes.loc[len(all_changes)] = [name, subname, row['time'], prev_row['time'], row['position'], row['orientation'], side, grasp_position]
            last_gripper = row['gripper']
        
    except Exception as e:
        print(f"Failed to process {csv_path}: {str(e)}")

# Process each CSV file found
for csv_file in csv_files:
    find_gripper_changes(csv_file)

# Save the collected data to a new CSV file
all_changes.to_csv("/media/ubb/4T/human_demonstration/arif_checked_process1/gripper_changes.csv", index=False)
'''


In [124]:
import os
import pandas as pd
import numpy as np
import open3d as o3d
import glob

# Path to the base directory where the subfolders are located
base_path = "/media/ubb/4T/human_demonstration/useful_data/xie_checked_process"

# Path for the merged tracker file
# csv_file = os.path.join(base_path, "bag","tracker_total_zed_modified.csv")
csv_files = glob.glob(os.path.join(base_path, "**/*tracker_total_zed_modified.csv"), recursive=True)
# Prepare a DataFrame to store all changes
all_changes = pd.DataFrame(columns=[
    'name', 'subname','index', 'time', 'pc_time',
    'original_pos_left', 'original_ori_left',
    'original_pos_right', 'original_ori_right',
    'grasp_pos_left', 'grasp_pos_right', 
    'left_gripper', 'right_gripper'
])

# def calculate_average_position(data, start_index, end_index, pos_column):
#     positions = []
#     for i in range(start_index, end_index + 1):
#         pos = data.loc[i, pos_column]
#         if isinstance(pos, str):
#             pos = eval(pos)
#         positions.append(np.array([pos['x'], pos['y'], pos['z']]))
#     return np.mean(np.array(positions), axis=0) if positions else np.array([0, 0, 0])

def position_to_nparray(position):
    return np.array([position['x'], position['y'], position['z']])

# Convert quaternion dictionary to numpy array
def quaternion_to_nparray(quaternion):
    return np.array([quaternion['w'], quaternion['x'], quaternion['y'], quaternion['z']])

# Function to calculate the extended point
def calculate_extended_point(position, quaternion, extension_vector):
    position_np = position_to_nparray(position)
    quaternion_np = quaternion_to_nparray(quaternion)
    R = o3d.geometry.get_rotation_matrix_from_quaternion(quaternion_np)
    extended_vector = np.dot(R, extension_vector)
    extended_point = position_np + extended_vector
    return extended_point.tolist()

# Function to process tracker data
def process_tracker_data(csv_path):
    global all_changes
    parts = csv_path.split(os.sep)
    print(parts)
    name = parts[-4]  # Adjust if necessary
    subname = parts[-3] 
    changes = pd.DataFrame(columns=all_changes.columns)
    try:
        # Read the CSV file
        data = pd.read_csv(csv_path)
        data['left_pos'] = data['left_pos'].apply(eval).apply(np.array)
        data['left_ori'] = data['left_ori'].apply(eval).apply(np.array)
        data['right_pos'] = data['right_pos'].apply(eval).apply(np.array)
        data['right_ori'] = data['right_ori'].apply(eval).apply(np.array)

        # Track the last gripper state to identify changes from 0 to 1
        last_gripper_left = 0
        last_gripper_right = 0

        for index, row in data.iterrows():
            changed_left = row['left_gripper'] == 1 and last_gripper_left == 0
            changed_right = row['right_gripper'] == 1 and last_gripper_right == 0

            pc_time = data.iloc[index - 8]['time'] if index >= 8 else row['time']
            if changed_left or changed_right:
            # Calculate extended positions
                # pc_time_index = max(0, index - 13)
                # average_pos_left = calculate_average_position(data, pc_time_index, index, 'left_pos')
                # average_pos_right = calculate_average_position(data, pc_time_index, index, 'right_pos')
                
                extension_vector_left = np.array([0.03, 0.03, 0.095])
                extension_vector_right = np.array([-0.03, 0.03, 0.095])
                grasp_pos_left = calculate_extended_point(row['left_pos'], row['left_ori'], extension_vector_left) if changed_left else None
                grasp_pos_right = calculate_extended_point(row['right_pos'], row['right_ori'], extension_vector_right) if changed_right else None
            
            # Append to all_changes DataFrame
                # all_changes.loc[len(all_changes)] = [
                #     name, subname, index, row['time'], pc_time, row['left_pos'], row['left_ori'],
                #     row['right_pos'], row['right_ori'], 
                #     grasp_pos_left, grasp_pos_right, 
                #     int(changed_left), int(changed_right)
                # ]
                changes.loc[len(changes)] = [
                    name, subname, index, row['time'], pc_time, row['left_pos'], row['left_ori'],
                    row['right_pos'], row['right_ori'], 
                    grasp_pos_left, grasp_pos_right, 
                    int(changed_left), int(changed_right)
                ]


            last_gripper_left = row['left_gripper']
            last_gripper_right = row['right_gripper']
        
        if not changes.empty:
            changes = changes.iloc[1:].reset_index(drop=True)
            all_changes = pd.concat([all_changes, changes])

    except Exception as e:
        print(f"Failed to process {csv_path}: {str(e)}")

for csv_file in csv_files:
    process_tracker_data(csv_file)


# Save the collected data to a new CSV file
output_csv_path = os.path.join(base_path, "gripper_changes_new.csv")
all_changes.to_csv(output_csv_path, index=False)
# print(f"Data saved to {output_csv_path}")

#  index from 0

['', 'media', 'ubb', '4T', 'human_demonstration', 'useful_data', 'xie_checked_process', 'ssshirt_blue', 'bag', 'tracker_total_zed_modified.csv']
['', 'media', 'ubb', '4T', 'human_demonstration', 'useful_data', 'xie_checked_process', 'lstshirt_white', 'bag', 'tracker_total_zed_modified.csv']
['', 'media', 'ubb', '4T', 'human_demonstration', 'useful_data', 'xie_checked_process', 'napkin_brown45', 'bag', 'tracker_total_zed_modified.csv']
['', 'media', 'ubb', '4T', 'human_demonstration', 'useful_data', 'xie_checked_process', 'napkin_fleek45', 'bag', 'tracker_total_zed_modified.csv']
['', 'media', 'ubb', '4T', 'human_demonstration', 'useful_data', 'xie_checked_process', 'napkin_grey45', 'bag', 'tracker_total_zed_modified.csv']
['', 'media', 'ubb', '4T', 'human_demonstration', 'useful_data', 'xie_checked_process', 'lstshirt_black', 'bag', 'tracker_total_zed_modified.csv']
['', 'media', 'ubb', '4T', 'human_demonstration', 'useful_data', 'xie_checked_process', 'napkin_pink45', 'bag', 'tracker_

In [126]:

import pandas as pd
import math

def filter_close_indices(csv_path):
    # Load the CSV file
    data = pd.read_csv(csv_path)

    # Create an empty list to store indices of rows to keep
    indices_to_drop = []

    # Get the 'index' column as a list
    indices = data['index'].tolist()
    
    # Iterate over the indices list and check differences
    for i in range(1, len(indices)):
        if abs(indices[i] - indices[i - 1]) <= 20:
            # Add both indices to the drop list if they are too close
            indices_to_drop.append(i - 1)
            indices_to_drop.append(i)

    # Remove duplicates from indices_to_drop
    indices_to_drop = list(set(indices_to_drop))

    # Drop the rows that are too close to each other
    filtered_data = data.drop(indices_to_drop).reset_index(drop=True)

    # Save the filtered data back to CSV or to a new file
    filtered_data.to_csv(csv_path.replace('.csv', '_filtered.csv'), index=False)
    print(f"Filtered data saved to {csv_path.replace('.csv', '_filtered.csv')}")

# Example usage
csv_file_path = "/media/ubb/4T/human_demonstration/useful_data/xie_checked_process/gripper_changes.csv"
filter_close_indices(csv_file_path)



Filtered data saved to /media/ubb/4T/human_demonstration/useful_data/xie_checked_process/gripper_changes_filtered.csv


according to the .csv select image

In [127]:
import os
import shutil
import pandas as pd

# Base directory for the operations
base_path = "/media/ubb/4T/human_demonstration/useful_data/xie_checked_process"
output_base = os.path.join(base_path, "grasp_point")

# Read the filtered CSV file
csv_path = os.path.join(base_path, "gripper_changes_filtered.csv")
data = pd.read_csv(csv_path)

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def copy_images(subfolder, pc_time):
    # Directories for images
    subfolders = ['bag_images', 'bag_side_images', 'bag_top_images']
    types = ['rgb', 'depth']

    for s_folder in subfolders:
        for img_type in types:
            # Source directory
            source_dir = os.path.join(base_path, subfolder, s_folder, img_type)
            # Destination directory
            dest_dir = os.path.join(output_base, subfolder, s_folder, img_type)
            ensure_dir(dest_dir)
            
            # Filename construction
            filename = f"{pc_time}.png"
            source_file = os.path.join(source_dir, filename)
            
            # Check if the file exists before attempting to copy
            if os.path.exists(source_file):
                shutil.copy(source_file, dest_dir)
                # print(f"Copied: {source_file} to {dest_dir}")
            else:
                print(f"File not found: {source_file}")

# Iterate over the dataframe
for index, row in data.iterrows():
    subfolder = row['subname']  # Ensure this column name matches your CSV structure
    pc_time = row['pc_time']
    copy_images(subfolder, pc_time)

print("Image copying completed.")


Image copying completed.


now sam2 from colab i can get the mask
prerequest: i have a rgb_for_mask folder, depth_for mask folder,mask folder -- then i apply the mask to the origal image

In [11]:
import os
from PIL import Image
import numpy as np

# Define the base directory containing all the subfolders
base_dir = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david_datacollect/baxter/drag_red45/pc"

# Iterate over each subfolder in the base directory
for subfolder in os.listdir(base_dir):
    subfolder_path = os.path.join(base_dir, subfolder)
    
    if os.path.isdir(subfolder_path):
        # Define subdirectories
        mask_dir = os.path.join(subfolder_path, "masks")
        rgb_dir = os.path.join(subfolder_path, "rgb")
        depth_dir = os.path.join(subfolder_path, "depth")
        rgb_mask_dir = os.path.join(subfolder_path, "rgb_mask")
        depth_mask_dir = os.path.join(subfolder_path, "depth_mask")

        # Create directories for masked images if they do not exist
        os.makedirs(rgb_mask_dir, exist_ok=True)
        os.makedirs(depth_mask_dir, exist_ok=True)

        # Get list of mask files
        mask_files = [f for f in os.listdir(mask_dir) if f.endswith('.png')]

        for mask_file in mask_files:
            mask_path = os.path.join(mask_dir, mask_file)
            rgb_path = os.path.join(rgb_dir, mask_file.replace('_mask.png', '.jpg'))
            depth_path = os.path.join(depth_dir, mask_file.replace('_mask.png', '.jpg'))

            # Load images
            mask = Image.open(mask_path).convert('L')  # Ensure mask is in grayscale
            rgb_image = Image.open(rgb_path)
            depth_image = Image.open(depth_path)

            # Convert images to arrays for masking
            mask_array = np.array(mask) / 255
            rgb_array = np.array(rgb_image)
            depth_array = np.array(depth_image)

            # Apply mask
            masked_rgb = np.dstack([(rgb_array[:, :, i] * mask_array).astype(np.uint8) for i in range(3)])
            masked_depth = (depth_array * mask_array).astype(depth_array.dtype)

            # Convert arrays back to images
            masked_rgb_image = Image.fromarray(masked_rgb)
            masked_depth_image = Image.fromarray(masked_depth)

            # Save the masked images
            masked_rgb_image.save(os.path.join(rgb_mask_dir, f'masked_{mask_file.replace("_mask.png", ".png")}'))
            masked_depth_image.save(os.path.join(depth_mask_dir, f'masked_{mask_file.replace("_mask.png", ".png")}'))

print("Finished processing all subfolders.")


Finished processing all subfolders.


create pointcloud see preate_graspvisulize.ipynb --genearate pointcloud resize
generate pointcloud for grasping point dataset

In [200]:
import os
import cv2
import open3d as o3d
import numpy as np

# Base directory containing all the subfolders
base_dir = "/media/ubb/4T/human_demonstration/grasp_point_top_all_sample"

# Camera intrinsics (example values, replace with actual intrinsics)
# intrinsics =(1070.1000, 1070.4301, 939.8100, 549.6520)
intrinsics = (1060.4000, 1060.0100, 946.0600, 496.6630)

def process_images_and_generate_pointclouds(rgb_dir, depth_dir, point_dir, intrinsics):
    if not os.path.exists(point_dir):
        os.makedirs(point_dir)

    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.endswith('.png')])
    depth_files = sorted([f for f in os.listdir(depth_dir) if f.endswith('.png')])
    processed_files = []
    for rgb_file, depth_file in zip(rgb_files, depth_files):
        rgb_image_path = os.path.join(rgb_dir, rgb_file)
        depth_image_path = os.path.join(depth_dir, depth_file)

        # Load and resize images
        color_raw = cv2.imread(rgb_image_path)
        color_raw = cv2.cvtColor(color_raw, cv2.COLOR_BGR2RGB)
        depth_raw = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)

        # Resize images to desired resolution (example: 1920x1080)
        color_hd = cv2.resize(color_raw, (1920, 1080), interpolation=cv2.INTER_CUBIC)
        depth_hd = cv2.resize(depth_raw, (1920, 1080), interpolation=cv2.INTER_NEAREST)

        # Convert images for point cloud generation
        depth_hd = o3d.geometry.Image(np.asarray(depth_hd))
        color_hd = o3d.geometry.Image(np.asarray(color_hd))

        # Generate RGBD image and point cloud
        

        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_hd, depth_hd, convert_rgb_to_intensity=False)
        fx, fy, cx, cy = intrinsics
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
            rgbd_image,
            o3d.camera.PinholeCameraIntrinsic(width=1920, height=1080, fx=fx, fy=fy, cx=cx, cy=cy)
        )
        pcd.transform([[-1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        if pcd.is_empty():
            print(f"No points were found in the point cloud for {rgb_file}")
        
        
        
        pointcloud_path = os.path.join(point_dir, rgb_file.replace('.png', '.pcd'))
        try:
            o3d.io.write_point_cloud(pointcloud_path, pcd)
            processed_files.append(rgb_file.replace('.png', '.pcd'))
            # print(f"Successfully saved point cloud: {pointcloud_path}")
        except Exception as e:
            print(f"Failed to save point cloud: {pointcloud_path}. Error: {str(e)}")
    return processed_files

missing_files=[]
# Process each subfolder
for subfolder in os.listdir(base_dir):
    subfolder_path = os.path.join(base_dir, subfolder)
    if os.path.isdir(subfolder_path):
        rgb_mask_dir = os.path.join(subfolder_path, "rgb_mask")
        depth_mask_dir = os.path.join(subfolder_path, "depth_mask")
        point_dir = os.path.join(subfolder_path, "pointcloud")
        processed_files = process_images_and_generate_pointclouds(rgb_mask_dir, depth_mask_dir, point_dir, intrinsics)
        if len(processed_files) < len(os.listdir(rgb_mask_dir)):
            expected_files = [f.replace('.png', '.pcd') for f in os.listdir(rgb_mask_dir) if f.endswith('.png')]
            missing_files.extend(list(set(expected_files) - set(processed_files)))
            print(f"Missing files in {subfolder}: {list(set(expected_files) - set(processed_files))}")

print("All missing files across subfolders:", missing_files)
        # process_images_and_generate_pointclouds(rgb_mask_dir, depth_mask_dir, point_dir, intrinsics)


All missing files across subfolders: []


pointcloud merge

In [201]:
import os
import open3d as o3d

# Define the base directories
base_dir_all = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud"
base_dir_top = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_top"
# base_dir_side = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_side"
base_dir_merge = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_merge2"


# Create the merge directory if it does not exist
os.makedirs(base_dir_merge, exist_ok=True)

def quaternion_to_rotation_matrix(quat):
    """Convert quaternion [w, x, y, z] to a 4x4 rotation matrix."""
    w, x, y, z = quat
    tx = 2.0 * x
    ty = 2.0 * y
    tz = 2.0 * z
    twx = tx * w
    twy = ty * w
    twz = tz * w
    txx = tx * x
    txy = ty * x
    txz = tz * x
    tyy = ty * y
    tyz = tz * y
    tzz = tz * z
 
    return np.array([[1.0 - (tyy + tzz), txy - twz, txz + twy, 0],
                     [txy + twz, 1.0 - (txx + tzz), tyz - twx, 0],
                     [txz - twy, tyz + twx, 1.0 - (txx + tyy), 0],
                     [0, 0, 0, 1]])
 
def transform_point_cloud(pcd, translation, quaternion):
    """Apply transformation defined by a translation and quaternion rotation to a point cloud."""
    # Create the transformation matrix
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    rotation_matrix_inversed = np.linalg.inv(rotation_matrix)
    #print(rotation_matrix)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix[:3, :3]
    transformation_matrix[:3, 3] = translation

    # transformation_matrix = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]] @ transformation_matrix
 
    # Transform the point cloud
    transformed_pcd = pcd.transform(transformation_matrix) # left multiplication
    #print(np.asarray(transformed_pcd.points))

    return transformed_pcd

def load_point_cloud(file_path):
    """Utility function to load a point cloud."""
    return o3d.io.read_point_cloud(file_path)

def merge_point_clouds(front_pcd, top_pcd):
    """Merge two point clouds and return the result."""
    return front_pcd + top_pcd

# Iterate over subdirectories in the 'all' directory
for subfolder in os.listdir(base_dir_all):
    subfolder_all = os.path.join(base_dir_all, subfolder)
    subfolder_top = os.path.join(base_dir_top, subfolder)
    subfolder_merge = os.path.join(base_dir_merge, subfolder)

    # Create subdirectory in merge directory
    os.makedirs(subfolder_merge, exist_ok=True)

    if os.path.isdir(subfolder_all) and os.path.isdir(subfolder_top):
        pointcloud_dir_all = os.path.join(subfolder_all, 'pointcloud')
        pointcloud_dir_top = os.path.join(subfolder_top, 'pointcloud')

        if os.path.exists(pointcloud_dir_all) and os.path.exists(pointcloud_dir_top):
            pointcloud_files_all = {f: os.path.join(pointcloud_dir_all, f) for f in os.listdir(pointcloud_dir_all) if f.endswith('.pcd')}
            pointcloud_files_top = {f: os.path.join(pointcloud_dir_top, f) for f in os.listdir(pointcloud_dir_top) if f.endswith('.pcd')}

        # Find matching point clouds and merge them
            for file_name, path_all in pointcloud_files_all.items():
                path_top = pointcloud_files_top.get(file_name)
                if path_top:
                    # Load point clouds
                    front_pcd = load_point_cloud(path_all)
                    top_pcd = load_point_cloud(path_top)
                    translation_zed = [0.139, -0.68, -1.525] # front modified leo final
                    # quaternion_zed =  [0.689, 0.128, 0.086, -0.708] # front modified leo
                    quaternion_zed =  [0.690, 0.126, 0.084, -0.708]  # front modified leo final
                    translation_zedtop = [-0.508, -0.313, -0.84] # top modified leo final
                    # quaternion_zedtop = [0.659, 0.608, 0.332, 0.293] # top modified leo
                    quaternion_zedtop = [0.652, 0.615, 0.329, 0.297] # top modified leo final
                    transformed_pcd_zed = transform_point_cloud(front_pcd, translation_zed, quaternion_zed)
                    transformed_pcd_zedtop = transform_point_cloud(top_pcd, translation_zedtop, quaternion_zedtop)
                    transformed_pcd_zedside = transform_point_cloud(top_pcd, translation_zedside, quaternion_zedside)
                    # Merge point clouds
                    merged_pcd_2 = merge_point_clouds(transformed_pcd_zed, transformed_pcd_zedtop)
                    merged_pcd_3 = merge_point_clouds(transformed_pcd_zed, transformed_pcd_zedtop,transformed_pcd_zedside)
                    # Save merged point cloud
                    output_path = os.path.join(subfolder_merge, file_name)
                    o3d.io.write_point_cloud(output_path, merged_pcd_2)
                    # print(f"Merged and saved point cloud to {output_path}")

print("Finished merging point clouds.")


Finished merging point clouds.


pointcloud downsample

In [202]:
import os
import open3d as o3d

def downsample_pointcloud(pcd_path, output_dir, voxel_size=0.0045):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load the point cloud
    pcd = o3d.io.read_point_cloud(pcd_path)
    # print(f"Original point count: {len(pcd.points)}")

    # Downsample the point cloud using voxel downsampling
    down_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
    # print(f"Downsampled point count: {len(down_pcd.points)}")

    # Save the downsampled point cloud
    downsampled_filename = os.path.basename(pcd_path).replace('.pcd', f'_downsampled_{voxel_size}.pcd')
    downsampled_path = os.path.join(output_dir, downsampled_filename)
    o3d.io.write_point_cloud(downsampled_path, down_pcd)
    # print(f"Downsampled point cloud saved to: {downsampled_path}")

def process_subfolders(base_dir, output_base_dir, voxel_size=0.0045):
    # Iterate through each subfolder in the base directory
    for subdir in os.listdir(base_dir):
        subdir_path = os.path.join(base_dir, subdir)
        if os.path.isdir(subdir_path):
            output_dir = os.path.join(output_base_dir, subdir)
            os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

            # Process each .pcd file in the subdirectory
            for file in os.listdir(subdir_path):
                if file.endswith('.pcd'):
                    pcd_path = os.path.join(subdir_path, file)
                    downsample_pointcloud(pcd_path, output_dir, voxel_size)

# Define base directories
base_dir = "/media/ubb/4T/human_demonstration/grasp_point_merge"
output_base_dir = "/media/ubb/4T/human_demonstration/grasp_point_merge_downsample"

# Process all subfolders
process_subfolders(base_dir, output_base_dir)


cvs merge

In [189]:
import os
import pandas as pd

# Define the base directory containing the subfolders
base_dir = "/media/ubb/Seagate Backup Plus Drive/human_demonstration/useful_data"

# Initialize an empty list to hold DataFrames
dataframes = []

# Iterate through each subdirectory in the base directory
for subdir in os.listdir(base_dir):
    subdir_path = os.path.join(base_dir, subdir)
    
    # Check if it's a directory
    if os.path.isdir(subdir_path):
        # Specify the path to the CSV file
        csv_path = os.path.join(subdir_path, "gripper_changes_filtered.csv")
        
        # Check if the CSV file exists
        if os.path.exists(csv_path):
            # Read the CSV file and append it to the list
            df = pd.read_csv(csv_path)
            dataframes.append(df)

# Concatenate all dataframes into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Define the path for the output CSV file
output_csv_path = "/media/ubb/Seagate Backup Plus Drive/human_demonstration/combined_gripper_changes.csv"

# Save the combined DataFrame to a CSV file
combined_df.to_csv(output_csv_path, index=False)

print(f"Combined CSV saved to: {output_csv_path}")


Combined CSV saved to: /media/ubb/Seagate Backup Plus Drive/human_demonstration/combined_gripper_changes.csv


In [190]:
import os
import pandas as pd

# Define the base directory containing the subfolders
csv_path1 = "/media/ubb/4T/human_demonstration/combined_gripper_changes.csv"
csv_path2 = "/media/ubb/backup1/human_demonstration/combined_gripper_changes.csv"
csv_path3 = "/media/ubb/Seagate Backup Plus Drive/human_demonstration/combined_gripper_changes.csv"


# Initialize an empty list to hold DataFrames
dataframes = []

df1 = pd.read_csv(csv_path1)
df2 = pd.read_csv(csv_path2)
df3 = pd.read_csv(csv_path3)

dataframes.append(df1)
dataframes.append(df2)
dataframes.append(df3)

# Concatenate all dataframes into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Define the path for the output CSV file
output_csv_path = "/media/ubb/4T/human_demonstration/combined_all_gripper_changes.csv"

# Save the combined DataFrame to a CSV file
combined_df.to_csv(output_csv_path, index=False)

print(f"Combined CSV saved to: {output_csv_path}")


Combined CSV saved to: /media/ubb/4T/human_demonstration/combined_all_gripper_changes.csv


csv and pointcloud check

In [191]:
import pandas as pd

# Load the combined CSV file
csv_path = "/media/ubb/4T/human_demonstration/combined_all_gripper_changes.csv"
df = pd.read_csv(csv_path)

# Filter rows where 'subname' contains 'lsshirt_bluedot' or 'lsshirt_darkblue'
filtered_df = df[df['subname'].str.contains('lsshirt_bluedot|lsshirt_darkblue', regex=True)]

# Define the path for the output CSV file
output_csv_path = "/media/ubb/4T/human_demonstration/combined_all_gripper_changes_leo.csv"

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv(output_csv_path, index=False)

print(f"Filtered CSV saved to: {output_csv_path}")


Filtered CSV saved to: /media/ubb/4T/human_demonstration/combined_all_gripper_changes_leo.csv


In [5]:
import pandas as pd
import os

# Load the CSV file
csv_path = "/media/ubb/My Passport/useful_data/combined_all_gripper_changes.csv"
df = pd.read_csv(csv_path)

# Extract pc_time values from the DataFrame and convert them to string for matching
pc_times = set(df['pc_time'].astype(str))

# Path to the directory containing pointcloud subfolders
base_dir = "/media/ubb/My Passport/grasp_point_robot/grasp_point_merge_downsample"

# Set to store the 'xxx' extracted from filenames
pcd_times = set()

# Iterate through each subfolder and each file within those subfolders
for subdir in os.listdir(base_dir):
    subdir_path = os.path.join(base_dir, subdir)
    if os.path.isdir(subdir_path):
        for filename in os.listdir(subdir_path):
            if filename.endswith('_downsampled_0.0045.pcd'):
                # Extract the 'xxx' part from 'masked_xxx_downsampled_0.0045.pcd'
                pcd_time = filename.split('_')[1]
                pcd_times.add(pcd_time)

# Find differences
missing_in_csv = pcd_times - pc_times
missing_in_pcd = pc_times - pcd_times

print("Missing in CSV, but present in pointclouds:", missing_in_csv)
print("Missing in pointclouds, but present in CSV:", missing_in_pcd)


Missing in CSV, but present in pointclouds: set()
Missing in pointclouds, but present in CSV: set()


delete extra pc

In [ ]:
missing_in_csv

In [210]:
import os

# Base directory where the pointclouds are stored
base_dir = "/media/ubb/My Passport/grasp_point_robot/grasp_point_merge_downsample"

# Iterate over all subfolders and delete the specified pointcloud files
for subdir in os.listdir(base_dir):
    subdir_path = os.path.join(base_dir, subdir)
    if os.path.isdir(subdir_path):
        #pointcloud_dir = os.path.join(subdir_path, 'pointcloud')  # Adjust if pointclouds are directly under subdir
        for filename in os.listdir(subdir_path):
            if filename.endswith('.pcd'):
                # Extract the 'xxx' part from 'masked_xxx_downsampled_0.0045.pcd'
                pcd_time = filename.split('_')[1]
                if pcd_time in missing_in_csv:
                    file_path = os.path.join(subdir_path, filename)
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")

print("Finished deleting pointcloud files.")


Deleted: /media/ubb/4T/human_demonstration/grasp_point_merge_downsample/ssshirt_blue/masked_1717506513903735808_downsampled_0.0045.pcd
Deleted: /media/ubb/4T/human_demonstration/grasp_point_merge_downsample/ssshirt_blue/masked_1717506509471247555_downsampled_0.0045.pcd
Deleted: /media/ubb/4T/human_demonstration/grasp_point_merge_downsample/ssshirt_blue/masked_1717506504639132401_downsampled_0.0045.pcd
Deleted: /media/ubb/4T/human_demonstration/grasp_point_merge_downsample/ssshirt_blue/masked_1717506497546558354_downsampled_0.0045.pcd
Deleted: /media/ubb/4T/human_demonstration/grasp_point_merge_downsample/pant_bluejean/masked_1717509591252639506_downsampled_0.0045.pcd
Deleted: /media/ubb/4T/human_demonstration/grasp_point_merge_downsample/pant_bluejean/masked_1717509587758704333_downsampled_0.0045.pcd
Deleted: /media/ubb/4T/human_demonstration/grasp_point_merge_downsample/pant_bluejean/masked_1717509584275542004_downsampled_0.0045.pcd
Deleted: /media/ubb/4T/human_demonstration/grasp_poi

delete extra row in csv

In [209]:
import pandas as pd

# Path to the CSV file
csv_path = "/media/ubb/4T/human_demonstration/combined_all_gripper_changes.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# # Display the number of rows before deletion for confirmation
# print("Number of rows before deletion:", len(df))

# # Delete the row where 'pc_time' is 1718819865018888256
# df = df[df['pc_time'] != 1718819865018888256]
delete_pc_times = {'1719142079084016418', '1719140552895662872', '1719156442063866156', '1718819865018888256', '1719059271281482711'}

# Delete the rows where 'pc_time' is in the set of delete_pc_times
df = df[~df['pc_time'].isin(delete_pc_times)]


# Display the number of rows after deletion to confirm the row is deleted
print("Number of rows after deletion:", len(df))

# Save the updated DataFrame back to CSV
df.to_csv(csv_path, index=False)

print(f"Rows with pc_times in {delete_pc_times} have been deleted.")


Number of rows after deletion: 6329
Rows with pc_times in {'1719142079084016418', '1719140552895662872', '1719156442063866156', '1718819865018888256', '1719059271281482711'} have been deleted.


another way, crop the pointcloud

In [17]:
def quaternion_to_rotation_matrix(quat):
    """Convert quaternion [w, x, y, z] to a 4x4 rotation matrix."""
    w, x, y, z = quat
    tx = 2.0 * x
    ty = 2.0 * y
    tz = 2.0 * z
    twx = tx * w
    twy = ty * w
    twz = tz * w
    txx = tx * x
    txy = ty * x
    txz = tz * x
    tyy = ty * y
    tyz = tz * y
    tzz = tz * z
 
    return np.array([[1.0 - (tyy + tzz), txy - twz, txz + twy, 0],
                     [txy + twz, 1.0 - (txx + tzz), tyz - twx, 0],
                     [txz - twy, tyz + twx, 1.0 - (txx + tyy), 0],
                     [0, 0, 0, 1]])
 
def transform_point_cloud(pcd, translation, quaternion):
    """Apply transformation defined by a translation and quaternion rotation to a point cloud."""
    # Create the transformation matrix
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    rotation_matrix_inversed = np.linalg.inv(rotation_matrix)
    #print(rotation_matrix)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix[:3, :3]
    transformation_matrix[:3, 3] = translation

    # transformation_matrix = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]] @ transformation_matrix
 
    # Transform the point cloud
    transformed_pcd = pcd.transform(transformation_matrix) # left multiplication
    #print(np.asarray(transformed_pcd.points))

    return transformed_pcd

In [25]:
import open3d as o3d
import numpy as np

def apply_aabb_cropping(pcd, min_bound, max_bound):
    # Create an axis-aligned bounding box
    aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound, max_bound=max_bound)
    # Crop the point cloud
    cropped_pcd = pcd.crop(aabb)
    return cropped_pcd

# Example usage
pcd_path = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/pointcloud/1719056738504144079.pcd"
pcd = o3d.io.read_point_cloud(pcd_path)

# Coordinates from CloudCompare (example, replace these with actual values from your screenshot)
x_min = -0.493813
x_max = 0.527173
y_min = -0.534935
y_max = 0.488354
z_min = 0.523
z_max = 1.45

min_bound = np.array([x_min, y_min, z_min])
max_bound = np.array([x_max, y_max, z_max])

# Crop the point cloud
cropped_pcd = apply_aabb_cropping(pcd, min_bound, max_bound)

# Optionally visualize the cropped point cloud
o3d.visualization.draw_geometries([cropped_pcd])


more specific see pointcloud_merge_threecamera.py

pointcloud registration global registration and local registration(icp) but do not work

In [20]:
from scipy.spatial.transform import Rotation as R
import os
import ast
import csv
import numpy as np
import cv2
import open3d as o3d
import os
import copy

def find_nearest_timestamp(target_timestamp, timestamps):
    timestamps = np.array(timestamps, dtype=np.int64)
    idx = np.abs(timestamps - target_timestamp).argmin()
    return timestamps[idx]

def load_point_cloud(file_path):
    return o3d.io.read_point_cloud(file_path)

def quaternion_to_rotation_matrix(quat):
    """Convert quaternion [w, x, y, z] to a 4x4 rotation matrix."""
    w, x, y, z = quat
    tx = 2.0 * x
    ty = 2.0 * y
    tz = 2.0 * z
    twx = tx * w
    twy = ty * w
    twz = tz * w
    txx = tx * x
    txy = ty * x
    txz = tz * x
    tyy = ty * y
    tyz = tz * y
    tzz = tz * z
 
    return np.array([[1.0 - (tyy + tzz), txy - twz, txz + twy, 0],
                     [txy + twz, 1.0 - (txx + tzz), tyz - twx, 0],
                     [txz - twy, tyz + twx, 1.0 - (txx + tyy), 0],
                     [0, 0, 0, 1]])
 
def transform_point_cloud(pcd, translation, quaternion):
    """Apply transformation defined by a translation and quaternion rotation to a point cloud."""
    # Create the transformation matrix
    rotation_matrix = quaternion_to_rotation_matrix(quaternion)
    rotation_matrix_inversed = np.linalg.inv(rotation_matrix)
    #print(rotation_matrix)
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix[:3, :3]
    transformation_matrix[:3, 3] = translation

    # transformation_matrix = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]] @ transformation_matrix
 
    # Transform the point cloud
    transformed_pcd = pcd.transform(transformation_matrix) # left multiplication
    #print(np.asarray(transformed_pcd.points))

    return transformed_pcd

def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh




def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def execute_icp(source, target, threshold=0.02, init_pose=None, icp_type=None):
    # Set initial alignment
    if init_pose is None:
        init_pose = np.identity(4)  # Identity matrix, no initial alignment
    
    # Compute point to point ICP
    if icp_type == 'point_to_point':
        reg_p2p = o3d.pipelines.registration.registration_icp(
            source, target, threshold, init_pose,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
        print(reg_p2p)
        transformation = reg_p2p.transformation
    elif icp_type == 'point_to_plane':
        # Ensure normals are estimated
        source.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        target.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        reg_p2plane = o3d.pipelines.registration.registration_icp(
            source, target, threshold, init_pose,
            o3d.pipelines.registration.TransformationEstimationPointToPlane())
        print(reg_p2plane)
        transformation = reg_p2plane.transformation
    
    return transformation

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    return result

def apply_aabb_cropping(pcd, min_bound, max_bound):
    # Create an axis-aligned bounding box
    aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound, max_bound=max_bound)
    # Crop the point cloud
    cropped_pcd = pcd.crop(aabb)
    return cropped_pcd

x_min = -0.493813
x_max = 0.527173
y_min = -0.534935
y_max = 0.488354
z_min = 0.523
z_max = 1.45

min_bound = np.array([x_min, y_min, z_min])
max_bound = np.array([x_max, y_max, z_max])

index = 130
pointcloud_folder = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/pointcloud"
pointcloud_top_folder = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/pointcloud_top"
pointcloud_side_folder = "/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/lipeng/napkin_pink30_5/pointcloud_side"
    
pointcloud_files = sorted(os.listdir(pointcloud_folder))
pointcloud_top_files = sorted(os.listdir(pointcloud_top_folder))
pointcloud_side_files = sorted(os.listdir(pointcloud_side_folder))
    
pointcloud_top_timestamps = [int(f.split('.')[0]) for f in pointcloud_top_files]
pointcloud_side_timestamps = [int(f.split('.')[0]) for f in pointcloud_side_files]

# for pointcloud_file in posintcloud_files:
pointcloud_file = pointcloud_files[index]
target_timestamp = int(pointcloud_file.split('.')[0])
        
nearest_top_timestamp = find_nearest_timestamp(target_timestamp, pointcloud_top_timestamps)
nearest_side_timestamp = find_nearest_timestamp(target_timestamp, pointcloud_side_timestamps)
        
pointcloud_path = os.path.join(pointcloud_folder, pointcloud_file)
top_pointcloud_path = os.path.join(pointcloud_top_folder, f"{nearest_top_timestamp}.pcd")
side_pointcloud_path = os.path.join(pointcloud_side_folder, f"{nearest_side_timestamp}.pcd")

# Load point clouds
front_pcd = load_point_cloud(pointcloud_path)
top_pcd = load_point_cloud(top_pointcloud_path)
side_pcd = load_point_cloud(side_pointcloud_path)

translation_zed = [0.139, -0.68, -1.525] # front modified leo final
# quaternion_zed =  [0.689, 0.128, 0.086, -0.708] # front modified leo
quaternion_zed =  [0.690, 0.126, 0.084, -0.708]  # front modified leo final

translation_zedtop = [-0.508, -0.313, -0.84] # top modified leo final
# quaternion_zedtop = [0.659, 0.608, 0.332, 0.293] # top modified leo
quaternion_zedtop = [0.652, 0.615, 0.329, 0.297] # top modified leo final


translation_zedside = [1.0, -1.030, -0.01] # side modified leo final
# quaternion_zedside =  [0.392,  -0.691, -0.578,  0.195] # side modified leo
# quaternion_zedside =  [  0.389, -0.683, -0.587,  0.193] 
quaternion_zedside =  [0.385, -0.687, -0.588,  0.186]  # side modified leo final

cropped_pcd_zed = apply_aabb_cropping(front_pcd, min_bound, max_bound)
cropped_pcd_top = apply_aabb_cropping(top_pcd, min_bound, max_bound)
cropped_pcd_side = apply_aabb_cropping(side_pcd, min_bound, max_bound)

transformed_pcd_zed = transform_point_cloud(cropped_pcd_zed, translation_zed, quaternion_zed)
transformed_pcd_zedtop = transform_point_cloud(cropped_pcd_top, translation_zedtop, quaternion_zedtop)
transformed_pcd_zedside = transform_point_cloud(cropped_pcd_side, translation_zedside, quaternion_zedside)
# transformed_pcd_zed = transform_point_cloud(front_pcd, translation_zed, quaternion_zed)
# transformed_pcd_zedtop = transform_point_cloud(top_pcd, translation_zedtop, quaternion_zedtop)
# transformed_pcd_zedside = transform_point_cloud(side_pcd, translation_zedside, quaternion_zedside)
initial_transform = np.eye(4)

voxel_size = 0.015
source_down, source_fpfh = preprocess_point_cloud(transformed_pcd_zed, voxel_size)
target_down, target_fpfh = preprocess_point_cloud(transformed_pcd_zedtop, voxel_size)

# result_ransac = execute_global_registration(source_down, target_down,source_fpfh, target_fpfh,voxel_size)
# print(result_ransac)
# initial_transform=result_ransac.transformation


# print("Initial alignment")
# evaluation = o3d.pipelines.registration.evaluate_registration(
#     transformed_pcd_zedtop, transformed_pcd_zed, 0.02, initial_transform)
# print(evaluation)

#  # Assuming some initial transformation or use the identity matrix
# aligned_transformation = execute_icp(transformed_pcd_zedtop, transformed_pcd_zed, init_pose=initial_transform, icp_type='point_to_point')
# transformed_pcd_zedtop.transform(aligned_transformation)


# draw_registration_result(transformed_pcd_zed, transformed_pcd_zedtop, initial_transform)
# draw_registration_result(front_pcd, top_pcd, trans_init)


vis = o3d.visualization.Visualizer()
vis.create_window(width=1920, height=1080)

origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])

# vis.add_geometry(transformed_pcd_zedside)
vis.add_geometry(transformed_pcd_zed)
vis.add_geometry(transformed_pcd_zedtop)
vis.run()
vis.destroy_window()

# merged_point_cloud = transformed_pcd_zed + transformed_pcd_zedtop + transformed_pcd_zedside
# output_path = os.path.join(base_dir, "merged_complete_pointcloud.pcd")
# o3d.io.write_point_cloud(output_path, merged_point_cloud)



:: Downsample with a voxel size 0.015.
:: Estimate normal with search radius 0.030.
:: Compute FPFH feature with search radius 0.075.
:: Downsample with a voxel size 0.015.
:: Estimate normal with search radius 0.030.
:: Compute FPFH feature with search radius 0.075.


avoid e-

future work. 3d gussian spliting to generate the [merged pointcloud]

others no necessary code

In [145]:
import os
from PIL import Image

# Path to the directory containing all subfolders
base_dir = "/media/ubb/4T/human_demonstration/grasp_point_all"

# Iterate over all subdirectories in the base directory
for subdir in os.listdir(base_dir):
    subdir_path = os.path.join(base_dir, subdir)
    
    # Check if it's a directory
    if os.path.isdir(subdir_path):
        # Create an 'rgb' subdirectory
        depth_dir = os.path.join(subdir_path, "depth")
        os.makedirs(depth_dir, exist_ok=True)
        
        # # Move all .png images to the 'rgb' directory and convert them to .jpg
        for filename in os.listdir(subdir_path):
            if filename.endswith('.png'):
                old_file_path = os.path.join(subdir_path, filename)
                new_filename = filename[:-4] + '.jpg'
                new_file_path = os.path.join(depth_dir, new_filename)
                
                # Rename and move the file to the 'depth' directory
                os.rename(old_file_path, new_file_path)
                # # Open the PNG image and convert it to JPEG
                # with Image.open(old_file_path) as img:
                #     # Create a new filename with .jpg extension
                #     new_filename = filename[:-4] + '.jpg'
                #     new_file_path = os.path.join(rgb_dir, new_filename)
                    
                #     # Convert and save the image
                #     img.save(new_file_path, 'JPEG')
                
                # # Optionally, delete the old .png file
                # os.remove(old_file_path)



print("Finished processing all subfolders.")


Finished processing all subfolders.


extra unneccessary code

In [217]:
import pandas as pd
from datetime import datetime, timezone, timedelta

# Load the CSV file
df = pd.read_csv('/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/processed_output_zed.csv')

# Function to convert ROS timestamp to readable datetime in UTC
def convert_timestamp_to_uk_time(timestamps_ns):
    # Convert nanosecond timestamp to seconds, adjust for timezone (UTC+0)
    return [datetime.fromtimestamp(ts / 1e9, timezone.utc) for ts in timestamps_ns]

# Convert all timestamps in the 'time' column to UK time
df['real_time'] = convert_timestamp_to_uk_time(df['time'])

# Save the updated DataFrame to a new CSV file
df.to_csv('/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/processed_output_zed_with_real_time.csv', index=False)


In [7]:
import open3d as o3d

def load_point_cloud(file_path):
    """Utility function to load a point cloud from a file."""
    return o3d.io.read_point_cloud(file_path)

# Load the point cloud from a file
pc = load_point_cloud('/home/ubb/Documents/Baxter_isaac/ROS2/src/experiment_recorder/data/baxter/david/baxter/838433/pointcloud_merge_threecamera/masked_1724954266649801567.pcd')

# Initialize the visualizer
vis = o3d.visualization.Visualizer()
vis.create_window('Point Cloud Visualization', width=1920, height=1080)

# Optionally add a coordinate frame for reference
origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])
# vis.add_geometry(origin)

# Add the loaded point cloud to the visualizer
vis.add_geometry(pc)

# Run the visualizer until the window is closed
vis.run()

# Destroy the window after closing
vis.destroy_window()
